# Portfolio Optimization Analysis
This notebook conducts a portfolio optimization analysis, focusing on comparing the performance of a short-sale constrained portfolio with a mean-variance optimal portfolio. We cover data loading, preparation, portfolio optimization, and performance comparison.

## Step 1: Import Necessary Libraries
Import all the required Python libraries for data manipulation, numerical analysis, and portfolio optimization.

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## Step 2: Load and Prepare the Data
Load the historical return data and prepare it by filtering for the relevant analysis period.

In [3]:
# Load the data
data_path = 'Data_2A_2023.xlsx'  # Update this path as necessary
data = pd.read_excel(data_path, index_col=0)

# Filter data for the relevant periods
data_until_2015 = data[data.index.year <= 2015]
data_from_2016 = data[data.index.year >= 2016]

## Step 3: Calculate Average Returns and Covariance Matrix
Compute the average returns using only positive returns and the covariance matrix using all available data up until 2015.

In [9]:
# Using all returns up until 2015
average_returns = data_until_2015.mean()
covariance_matrix = data_until_2015.cov()

sum_avg = sum(average_returns) / 5

# Calculate average returns using only the days when the returns were positive
positive_average_returns = data_until_2015[data_until_2015 > 0].mean()
sum_avg_pos = sum(positive_average_returns) / 5

average_returns

AAPL    0.124552
DIS     0.056689
GE      0.019221
GS      0.050193
MSFT    0.027795
dtype: float64

In [7]:
sum_avg

0.0556897326187464

In [10]:
sum_avg_pos

1.5357618917167188

## Step 4: Define Portfolio Variance Function
Define a function to calculate the portfolio variance based on the given weights and the covariance matrix.


In [4]:
def portfolio_variance(weights, covariance_matrix):
    return weights.T @ covariance_matrix @ weights

## Step 5: Optimize Portfolios
Perform portfolio optimization to find the weights that minimize variance while meeting a specified target return. This section includes optimizations for short-sale constrained portfolio, a mean-variance optimal portfolio and minimum variance portfolio.

In [21]:
# Calculate the target return as the mean of positive average returns
target_return = np.mean(positive_average_returns)


# Constraints: weights sum to 1
constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
    {'type': 'eq', 'fun': lambda weights: weights @ positive_average_returns - target_return}  # Achieve target return

    ]

# Initial guess
initial_guess = np.full(len(positive_average_returns), 1 / len(positive_average_returns))

# Perform optimization using scipy's minimize
optimized_result = minimize(
    fun=portfolio_variance,
    x0=initial_guess,
    args=(covariance_matrix,),
    method='SLSQP',
    constraints=constraints,
)

optimized_weights = optimized_result.x

optimized_weights

array([0.23534618, 0.24438574, 0.18070111, 0.12395144, 0.21561552])

In [22]:
# Define the target return for the mean-variance portfolio
target_return_mv = np.mean(average_returns)  # Example: using the mean of average returns as the target

# Constraints: Weights sum to 1 and achieve the target return
constraints_mv = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights equals 1
    {'type': 'eq', 'fun': lambda weights: weights @ average_returns - target_return_mv}  # Achieve target return
]

# No bounds for short selling, allowing negative weights
bounds_mv = [(None, None) for _ in average_returns]  # This allows for short selling

# Optimization for the mean-variance optimal portfolio
mv_optimized_result = minimize(
    fun=portfolio_variance,
    x0=initial_guess,
    args=(covariance_matrix,),
    method='SLSQP',
    constraints=constraints_mv,
    bounds=bounds_mv
)

mv_optimized_weights = mv_optimized_result.x

mv_optimized_weights

array([0.1960708 , 0.33860056, 0.19563236, 0.03650267, 0.23319362])

In [27]:
# Constraints: weights sum to 1; bounds prevent short selling
constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]

# Initial guess
initial_guess = np.full(len(average_returns), 1 / len(average_returns))

# Optimization
optimized_result_minimum = minimize(
    fun=portfolio_variance,
    x0=initial_guess,
    args=(covariance_matrix,),
    method='SLSQP',
    constraints=constraints,
)

optimized_weights_minimum = optimized_result_minimum.x

optimized_weights_minimum

array([0.09033722, 0.2887378 , 0.29372827, 0.02036226, 0.30683445])

In [23]:
def calculate_annualized_performance(data, weights):
    portfolio_returns = data @ weights
    annualized_returns = portfolio_returns.mean() * 252
    annualized_variance = portfolio_returns.var() * 252
    return annualized_returns, annualized_variance

## Step 6: Calculate Performance from 2016 Onwards
Use the optimized weights to calculate and compare the annualized performance of both portfolios based on data from 2016 onwards.


In [29]:
# Short-Sale Constrained Portfolio
annualized_returns_constrained, annualized_variance_constrained = calculate_annualized_performance(data_from_2016, optimized_weights)

# Mean-Variance Optimal Portfolio
# Assuming `mv_optimized_weights` is defined
annualized_returns_mv, annualized_variance_mv = calculate_annualized_performance(data_from_2016, mv_optimized_weights)

annualized_returns_minimum, annualized_variance_minimum = calculate_annualized_performance(data_from_2016, optimized_weights_minimum)

## Conclusion
Summarize the findings from the portfolio optimization and performance comparison. Discuss the implications of short-sale constraints on portfolio performance.


In [30]:
print(f"Short-Sale Constrained Portfolio:\nAnnualized Returns: {annualized_returns_constrained}\nAnnualized Variance: {annualized_variance_constrained}\n")
print(f"Mean-Variance Optimal Portfolio:\nAnnualized Returns: {annualized_returns_mv}\nAnnualized Variance: {annualized_variance_mv}\n")
print(f"Minimum Variance Optimal Portfolio:\nAnnualized Returns: {annualized_returns_minimum}\nAnnualized Variance: {annualized_variance_minimum}\n")

Short-Sale Constrained Portfolio:
Annualized Returns: 13.865884236684565
Annualized Variance: 570.5643561487298

Mean-Variance Optimal Portfolio:
Annualized Returns: 11.948659031689811
Annualized Variance: 573.1603783731881

Minimum Variance Optimal Portfolio:
Annualized Returns: 9.88613475925987
Annualized Variance: 598.4660214270266

